# import stuff

In [1]:
from os import environ
from dotenv import load_dotenv
import numpy as np
import pandas as pd
from riotwatcher import LolWatcher, ApiError

load_dotenv("../")

RIOT_API = environ["RIOT_API"]
lol_watcher = LolWatcher(RIOT_API)
my_region = "na1"
NA = "na1"

# initialize stuff

In [2]:
yogurtsauce = pd.DataFrame.from_dict(
    lol_watcher.summoner.by_name(NA, "yogurtsauce"), orient="index"
).T

ranked = lol_watcher.league.by_summoner(NA, yogurtsauce.id[0])
versions = lol_watcher.data_dragon.versions_for_region(NA)
champions_version = versions["n"]["champion"]
summoner_spells_version = versions["n"]["summoner"]
items_version = versions["n"]["item"]
current_champ_list = lol_watcher.data_dragon.champions(champions_version)
my_matches = lol_watcher.match.matchlist_by_puuid(NA, yogurtsauce.puuid[0])
current_items = lol_watcher.data_dragon.items(items_version)
mastery_version = versions["n"]["mastery"]

# league

In [29]:
class League:
    def __init__(self, region) -> None:
        self.versions = self.Versions(region)
        self.champions = self.get_champions(region)

    class Versions:
        def __init__(self, region) -> None:
            self.versions_ = lol_watcher.data_dragon.versions_for_region(region)
            self.versions_item = self.versions_["n"]["item"]
            self.versions_rune = self.versions_["n"]["rune"]
            self.versions_mastery = self.versions_["n"]["mastery"]
            self.versions_summoner = self.versions_["n"]["summoner"]
            self.versions_champion = self.versions_["n"]["champion"]

    def get_champions(self, region):
        champions = (
            pd.DataFrame.from_dict(
                lol_watcher.data_dragon.champions(
                    self.Versions(region).versions_champion
                )
            )
            .reset_index()
            .rename(columns={"index": "champion"})
        )

        champions = (
            pd.concat([champions, pd.json_normalize(champions.data)], axis=1)
            .drop(columns=["type", "format", "version", "data", "name", "id"])
            .rename(
                {
                    "key": "champion_id",
                    "tags": "roles",
                    "partype": "resource",
                    # info
                    "info.attack": "attack",
                    "info.defense": "defense",
                    "info.magic": "magic",
                    "info.difficulty": "difficulty",
                    # stats
                    # hp
                    "stats.hp": "hp",
                    "stats.hpperlevel": "hp_level",
                    "stats.hpregen": "hp_regen",
                    "stats.hpregenperlevel": "hpregen_level",
                    # mp
                    "stats.mp": "mp",
                    "stats.mpperlevel": "mp_level",
                    "stats.mpregen": "mp_regen",
                    "stats.mpregenperlevel": "mpregen_level",
                    # ad
                    "stats.attackdamage": "attack",
                    "stats.attackdamageperlevel": "attack_level",
                    # crit
                    "stats.crit": "crit",
                    "stats.critperlevel": "crit_level",
                    # armour
                    "stats.armor": "armor",
                    "stats.armorperlevel": "armor_level",
                    # attack speed
                    "stats.attackspeed": "atk_spd",
                    "stats.attackspeedperlevel": "atk_spd_level",
                    # ms
                    "stats.movespeed": "move_spd",
                    # attack range
                    "stats.attackrange": "atk_range",
                    # spellblock
                    "stats.spellblock": "spellblock",
                    "stats.spellblockperlevel": "spellblock_level",
                },
                axis=1,
            )
        )
        
        champions.insert(0, "champion_id", champions.pop("champion_id"))
        
        champions.resource = champions.resource.astype("category")
        
        return champions


League("na1").champions.head(1)


,champion_id,champion,title,blurb,roles,resource,attack,defense,magic,difficulty,...,hp_regen,hpregen_level,mp_regen,mpregen_level,crit,crit_level,attack,attack_level,atk_spd_level,atk_spd
0,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"[Fighter, Tank]",Blood Well,8,4,3,4,...,3.0,1.0,0.0,0.0,0,0,60,5.0,2.5,0.651


# summoner

In [3]:
class Summoner:
    def __init__(self, region, summoner_name):
        self.region = region
        self.name = summoner_name
        self.data = pd.DataFrame.from_dict(
            lol_watcher.summoner.by_name(self.region, self.name), orient="index"
        ).T
        self.id = self.data.id[0]
        self.account_id = self.data.accountId[0]
        self.puuid = self.data.puuid[0]
        self.profile_icon_id = self.data.profileIconId[0]
        self.revision_date = self.data.revisionDate[0]
        self.level = self.data.summonerLevel[0]
        self.matches_ = lol_watcher.match.matchlist_by_puuid(self.region, self.puuid)


yogurtsauce = Summoner("na1", "yogurtsauce")

# champions

In [20]:
champions = (
    pd.DataFrame.from_dict(current_champ_list)
    .reset_index()
    .rename(columns={"index": "champion"})
)

champions = (
    pd.concat([champions, pd.json_normalize(champions.data)], axis=1)
    .drop(columns=["type", "format", "version", "data", "name", "id"])
    .rename(
        {
            "key": "champion_id",
            "tags": "roles",
            "partype": "resource",
            # info
            "info.attack": "attack",
            "info.defense": "defense",
            "info.magic": "magic",
            "info.difficulty": "difficulty",
            # stats
            # hp
            "stats.hp": "hp",
            "stats.hpperlevel": "hp_level",
            "stats.hpregen": "hp_regen",
            "stats.hpregenperlevel": "hpregen_level",
            # mp
            "stats.mp": "mp",
            "stats.mpperlevel": "mp_level",
            "stats.mpregen": "mp_regen",
            "stats.mpregenperlevel": "mpregen_level",
            # ad
            "stats.attackdamage": "attack",
            "stats.attackdamageperlevel": "attack_level",
            # crit
            "stats.crit": "crit",
            "stats.critperlevel": "crit_level",
            # armour
            "stats.armor": "armor",
            "stats.armorperlevel": "armor_level",
            # attack speed
            "stats.attackspeed": "atk_spd",
            "stats.attackspeedperlevel": "atk_spd_level",
            # ms
            "stats.movespeed": "move_spd",
            # attack range
            "stats.attackrange": "atk_range",
            # spellblock
            "stats.spellblock": "spellblock",
            "stats.spellblockperlevel": "spellblock_level",
        },
        axis=1,
    )
)

champions.insert(0, "champion_id", champions.pop("champion_id"))
# champions[["primary_role", "secondary_role"]] = pd.DataFrame(
#     champions.tags.tolist(), index=champions.index
# )
# champions = champions.drop(columns="tags")
# champions.primary_role = champions.primary_role.astype("category")
# champions.secondary_role = champions.secondary_role.astype("category")
champions.resource = champions.resource.astype("category")

,champion_id,champion,title,blurb,roles,resource,attack,defense,magic,difficulty,...,hp_regen,hpregen_level,mp_regen,mpregen_level,crit,crit_level,attack,attack_level,atk_spd_level,atk_spd
0,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"[Fighter, Tank]",Blood Well,8,4,3,4,...,3.0,1.0,0.0,0.0,0,0,60,5.0,2.5,0.651


# match history

In [15]:
def match_history(username, region, n_games=5):
    i = 0
    j = 0
    user = Summoner(region, username)
    matches = pd.DataFrame()

    while j < n_games:
        try:
            last_match = user.matches_[j]
            match_details = lol_watcher.match.by_id(region, last_match)
            match_players = match_details["info"]["participants"]
            details = []

            for player in match_players:
                dict = {}
                dict["ign"] = player["summonerName"]
                dict["level"] = player["summonerLevel"]
                dict["champion_id"] = player["championId"]
                dict["champion"] = player["championName"]
                dict[
                    "mastery_points"
                ] = lol_watcher.champion_mastery.by_summoner_by_champion(
                    region, player["summonerId"], player["championId"]
                )[
                    "championPoints"
                ]
                dict["team"] = player["teamId"]
                dict["win"] = player["win"]

                details.append(dict)

            details = pd.DataFrame.from_dict(details)
            details = details.merge(
                champions[
                    [
                        "champion",
                        "attack",
                        "defense",
                        "magic",
                        "difficulty",
                        "partype",
                    ]
                ]
            )

            details.team = details.team.astype("category")

            matches = pd.concat([matches, details], ignore_index=True)

            i += 1
            j += 1
        except:
            j += 1

    return matches


test = match_history("yogurtsauce", "na1", 1)

In [16]:
test

""


In [7]:
last_match = lol_watcher.match.by_id("na1", yogurtsauce.matches_[0])

In [8]:
wins = test.where(test.win == True)
wins = wins[~wins["ign"].isnull()]

losses = test.where(test.win == False)
losses = losses[~losses["ign"].isnull()]

AttributeError: 'DataFrame' object has no attribute 'win'

In [ ]:
wins

,ign,level,champion_id,champion,mastery_points,team,win,primary_role,secondary_role,info.attack,info.defense,info.magic,info.difficulty,partype
5,itsdango,402.0,517.0,Sylas,49330.0,200,True,Mage,Assassin,3.0,4.0,8.0,5.0,Mana
6,botfrag,367.0,136.0,AurelionSol,11574.0,200,True,Mage,NaN,2.0,3.0,8.0,7.0,Mana
7,FATHER GYOZA,496.0,164.0,Camille,17097.0,200,True,Fighter,Tank,8.0,6.0,3.0,4.0,Mana
8,HuyAnh Maii,681.0,876.0,Lillia,16379.0,200,True,Fighter,Mage,0.0,2.0,10.0,8.0,Mana
9,A Real Struggle,567.0,127.0,Lissandra,68822.0,200,True,Mage,NaN,2.0,5.0,8.0,6.0,Mana


In [ ]:
wins["champion1"] = wins["champion"].str.split().str[0]
wins["champion2"] = wins["champion"].str.split().str[1]
wins["champion2"] = wins["champion"].str.split().str[1]
wins["champion2"] = wins["champion"].str.split().str[1]
wins["champion2"] = wins["champion"].str.split().str[1]